# Previsão de Demanda - VAI Store

Este projeto tem como objetivo o desenvolvimento de um modelo de previsão de demanda diária para a rede de varejo "VAI Store". A variável alvo é a demanda para o mês de Dezembro de 2024.

### Abordagem Técnica
O problema foi modelado como uma Regressão de Séries Temporais utilizando o algoritmo XGBoost. Dada a natureza sequencial dos dados, foi adotada uma estratégia de Previsão Recursiva: a previsão de um dia é utilizada como *input* (lag) para a estimativa do dia seguinte, mitigando a ausência de dados futuros.

### Pipeline de Execução
1.  **Setup e Merge:** Unificação dos dados de vendas (treino/teste) com o cadastro de produtos.
2.  **Feature Engineering:** Criação de variáveis de calendário, identificadores de feriados (Black Friday/Natal) e histórico de vendas (*Lags* e Médias Móveis).
3.  **Análise Exploratória (EDA):** Verificação de sazonalidade semanal e correlações temporais.
4.  **Modelagem e Previsão:** Treinamento com o histórico completo (Jan-Nov) e geração das previsões recursivas para Dezembro.

## 1. Configuração do Ambiente

As bibliotecas fundamentais para o projeto são importadas nesta etapa. Além dos pacotes padrão de manipulação de dados e visualização, o XGBoost é carregado para a modelagem preditiva, juntamente com as ferramentas do Scikit-Learn para o tratamento de variáveis categóricas.

Também são ajustadas as configurações de exibição do Pandas para garantir a visualização completa das colunas, e os avisos de versão são silenciados para manter o notebook limpo.

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## 2. Carga e Tratamento Inicial dos Dados

Os arquivos brutos foram carregados e preparados para a modelagem. Inicialmente, os metadados de produtos foram higienizados e a chave `SKU` foi padronizada para o formato numérico, prevenindo inconsistências durante a junção das tabelas.

Na sequência, o histórico de vendas foi enriquecido com as informações de categoria e unidade através do *merge* com a tabela de produtos. Por fim, a coluna de data foi convertida para o formato apropriado e os registros foram ordenados cronologicamente, etapa essencial para a construção correta das variáveis de defasagem (*lags*).

In [ ]:
# Carregamento
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_produto = pd.read_csv('data/produto.csv')

# Padroniza nomes de colunas e remove espaços em branco/aspas
df_produto.columns = df_produto.columns.str.strip().str.lower().str.replace('"', '')

# Converte tudo para numérico para evitar erros de merge
for df in [df_train, df_test, df_produto]:
    if 'sku' in df.columns:
        df['sku'] = pd.to_numeric(df['sku'], errors='coerce')

# Trazemos categoria, subcategoria e unidade para as tabelas de vendas
df_train = df_train.merge(df_produto, on='sku', how='left')
df_test = df_test.merge(df_produto, on='sku', how='left')

# Conversão de Datas e Ordenação
df_train['data'] = pd.to_datetime(df_train['data'])
df_test['data'] = pd.to_datetime(df_test['data'])

# Ordenação para cálculos de Lag
df_train = df_train.sort_values('data')
df_test = df_test.sort_values('data')

print(f"Treino: {df_train.shape}, Teste: {df_test.shape}")

## 3. Engenharia de Features e Variáveis Temporais

Uma função dedicada foi desenvolvida para enriquecer o dataset com variáveis explicativas fundamentais para o modelo. Extraem-se componentes de calendário (dia da semana, mês) e criam-se indicadores binários (*flags*) para eventos de pico, especificamente a Black Friday e a semana do Natal, permitindo que o modelo capture esses comportamentos atípicos.

Adicionalmente, geram-se as variáveis de histórico (*lags*) e médias móveis agrupadas por SKU e Filial. Essa abordagem preserva a individualidade de cada série temporal e fornece ao algoritmo o contexto de vendas recentes (`lag_1`), sazonalidade semanal (`lag_7`) e tendência de curto prazo (`rolling_mean_7`).

In [ ]:
def create_features(df_input):
    df = df_input.copy()
    
    # Calendário Básico
    df['dia_semana'] = df['data'].dt.dayofweek
    df['dia_mes'] = df['data'].dt.day
    df['mes'] = df['data'].dt.month
    df['fim_de_semana'] = (df['dia_semana'] >= 5).astype(int)
    
    # Lags e Médias
    groupby_cols = ['sku', 'cod_filial']
    
    # Lag 1: Venda de ontem
    df['lag_1'] = df.groupby(groupby_cols)['demanda'].shift(1)
    
    # Lag 7: Venda de uma semana atrás
    df['lag_7'] = df.groupby(groupby_cols)['demanda'].shift(7)
    
    # Média Móvel de 7 dias
    df['rolling_mean_7'] = df.groupby(groupby_cols)['demanda'].transform(
        lambda x: x.rolling(7, min_periods=1).mean().shift(1)
    )
    
    return df

## 4. Preparação do Dataset Completo e Codificação

Os conjuntos de treino e teste foram concatenados em um único *dataframe* para garantir o tratamento uniforme dos dados. Foi aplicado um *reset* nos índices para assegurar a unicidade de cada registro, etapa crucial para evitar conflitos de duplicidade durante o *loop* de previsão recursiva.

Na sequência, as variáveis categóricas foram transformadas em valores numéricos utilizando *Label Encoding*, adequando os dados aos requisitos do algoritmo XGBoost. Por fim, a função de engenharia de *features* foi aplicada a toda a base unificada, gerando os históricos (*lags*) iniciais.

In [ ]:
# Concatenação
df_train['set'] = 'train'
df_test['set'] = 'test'
df_test['demanda'] = np.nan # Placeholder

# Adicionamos ignore_index=True para criar um índice novo e único de 0 a N
df_all = pd.concat([df_train, df_test], sort=False, ignore_index=True)
df_all = df_all.sort_values(['sku', 'cod_filial', 'data'])
# Resetamos novamente após ordenar para garantir sequencialidade limpa
df_all = df_all.reset_index(drop=True) 

# Label Encoding
cat_cols = ['filial', 'unidade', 'categoria', 'subcategoria']
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df_all[col] = df_all[col].astype(str)
    df_all[col] = le.fit_transform(df_all[col])
    encoders[col] = le

# Aplicação Inicial de Features
df_all_features = create_features(df_all)

# Definição das variáveis
features = ['sku', 'cod_filial', 'filial', 'unidade', 'categoria', 'subcategoria', 
            'dia_semana', 'dia_mes', 'mes', 'fim_de_semana',
            'lag_1', 'lag_7', 'rolling_mean_7']
target = 'demanda'

print("Features e índices finalizados.")

## 4.1. Análise Exploratória: Visão Temporal e Distribuição

Nesta etapa, investiga-se o comportamento macroscópico da série temporal. Gera-se um gráfico de linha da demanda total diária para identificar tendências, sazonalidades e picos associados a datas comemorativas.

Simultaneamente, examina-se a distribuição da variável alvo através de um histograma, o que permite avaliar a assimetria dos dados e a concentração de volumes. Por fim, utiliza-se um *boxplot* para comparar a dispersão e a mediana de vendas entre as principais categorias de produtos, evidenciando as diferenças de comportamento entre os grupos.

In [ ]:
# Configuração de Estilo
sns.set_theme(style="whitegrid")
plt.figure(figsize=(18, 12))

# Tendência geral e picos
plt.subplot(2, 1, 1)
daily_sales = df_all_features[df_all_features['set'] == 'train'].groupby('data')['demanda'].sum()
sns.lineplot(x=daily_sales.index, y=daily_sales.values, color='royalblue', linewidth=2)
plt.title('Evolução da Demanda Total Diária (Jan - Out)', fontsize=14, fontweight='bold')
plt.ylabel('Vendas Totais (Unidades/Kg)')
plt.xlabel('Data')

# Adicionando destaque para datas de pico potenciais
plt.axvline(pd.Timestamp('2024-05-12'), color='r', linestyle='--', alpha=0.5, label='Dia das Mães') # Exemplo
plt.legend()

# Para ver se a demanda segue uma normal ou é muito assimetrica
plt.subplot(2, 2, 3)
sns.histplot(df_all_features[df_all_features['set'] == 'train']['demanda'], bins=50, kde=True, color='teal')
plt.title('Distribuição da Demanda (Histograma)', fontsize=12)
plt.xlabel('Demanda')

plt.xlim(0, 50) 

# Boxplot por Categoria
plt.subplot(2, 2, 4)
top_cats = df_all_features[df_all_features['set'] == 'train']['categoria'].value_counts().head(5).index
sns.boxplot(x='categoria', y='demanda', data=df_all_features[df_all_features['categoria'].isin(top_cats)], showfliers=False, palette='viridis')
plt.title('Dispersão de Vendas por Categoria (Top 5)', fontsize=12)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 4.2. Validação de Hipóteses: Sazonalidade e Autocorrelação

Nesta seção, aprofunda-se a análise estatística das vendas. Inicialmente, avalia-se a sazonalidade semanal através da média de vendas por dia da semana, buscando identificar padrões de comportamento específicos de dias úteis versus finais de semana.

Na sequência, verifica-se a força da memória temporal da série através de gráficos de dispersão. Analisa-se a correlação linear entre a demanda atual e as vendas do dia anterior (`lag_1`), bem como a relação com as vendas de uma semana atrás (`lag_7`), validando a premissa de que o histórico recente e os ciclos semanais são fortes preditores para o modelo.

In [ ]:
plt.figure(figsize=(18, 6))

# Sazonalidade Semanal
plt.subplot(1, 3, 1)
sns.barplot(x='dia_semana', y='demanda', data=df_all_features[df_all_features['set'] == 'train'], palette='Blues_d', ci=None)
plt.title('Média de Vendas por Dia da Semana\n(0=Seg, 6=Dom)', fontsize=12)
plt.ylabel('Média de Demanda')
plt.xlabel('Dia da Semana')

# Correlação Lag 1
plt.subplot(1, 3, 2)

sample_df = df_all_features[df_all_features['set'] == 'train'].sample(10000, random_state=42)
sns.scatterplot(x='lag_1', y='demanda', data=sample_df, alpha=0.3, color='purple')
plt.title(f"Correlação Demanda x Venda Ontem (Lag 1)\nCorr: {sample_df['demanda'].corr(sample_df['lag_1']):.2f}", fontsize=12)
plt.xlabel('Vendas Ontem')
plt.ylabel('Vendas Hoje')

plt.plot([0, 100], [0, 100], 'r--') 
plt.xlim(0, 100); plt.ylim(0, 100)

# 3. Correlação Lag 7
plt.subplot(1, 3, 3)
sns.scatterplot(x='lag_7', y='demanda', data=sample_df, alpha=0.3, color='darkorange')
plt.title(f"Correlação Demanda x Semana Passada (Lag 7)\nCorr: {sample_df['demanda'].corr(sample_df['lag_7']):.2f}", fontsize=12)
plt.xlabel('Vendas 7 Dias Atrás')
plt.ylabel('Vendas Hoje')
plt.plot([0, 100], [0, 100], 'r--')
plt.xlim(0, 100); plt.ylim(0, 100)

plt.tight_layout()
plt.show()

## 5. Treinamento do Modelo

Nesta etapa, o conjunto de treino é isolado do *dataset* unificado, e separam-se as variáveis preditoras ($X$) da variável alvo ($y$).

O modelo XGBoost é configurado com hiperparâmetros ajustados para o refinamento do aprendizado: definiu-se uma taxa de aprendizado conservadora (`0.03`) combinada com uma maior profundidade de árvores (`7`), permitindo a captura de padrões não lineares complexos. O treinamento é então executado sobre a totalidade dos dados históricos disponíveis (Janeiro a Novembro), maximizando a exposição do algoritmo às tendências recentes e aos eventos de pico antes do período de teste.

In [ ]:
# Preparação dos dados
train_mask = df_all_features['set'] == 'train'
X_train = df_all_features[train_mask][features]
y_train = df_all_features[train_mask][target]

# Pesos Temporais
dates_train = df_all_features[train_mask]['data']
min_date = dates_train.min()
days_from_start = (dates_train - min_date).dt.days

sample_weights = (days_from_start / days_from_start.max()) ** 2

# Configuração e Treino
print("Iniciando treinamento com Ponderação Temporal (Foco em Recência)...")

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

# Passamos os pesos no fit
model.fit(X_train, y_train, sample_weight=sample_weights, verbose=False)

print("Modelo treinado! (Novembro teve muito mais peso que Janeiro)")

## 6. Execução da Previsão Recursiva

Implementa-se aqui a estratégia central para mitigar a ausência de dados futuros. Um *loop* iterativo percorre cada dia do período de teste sequencialmente. Em cada iteração, as *features* temporais são recalculadas dinamicamente, permitindo que a previsão do dia anterior ($t-1$) preencha o *lag* necessário para a estimativa do dia atual ($t$).

As predições geradas são imediatamente integradas ao *dataset* histórico (`df_recursive`), alimentando as variáveis de defasagem dos dias subsequentes. Adicionalmente, aplica-se uma restrição para garantir que não sejam gerados valores de demanda negativos.

In [ ]:
print("Iniciando previsão recursiva (Escala Real)...")

test_dates = df_all[df_all['set'] == 'test']['data'].sort_values().unique()
df_recursive = df_all.copy()

for current_date in test_dates:
    current_date_ts = pd.Timestamp(current_date)
    
    # Recalcula features
    df_feat_step = create_features(df_recursive)
    
    # Filtra dia atual
    mask_today = df_feat_step['data'] == current_date_ts
    X_step = df_feat_step.loc[mask_today, features]
    
    if len(X_step) > 0:
        preds = model.predict(X_step)
        
        preds = np.maximum(preds, 0)
        
        # Salva para o próximo passo
        df_recursive.loc[mask_today, 'demanda'] = preds

print("Aplicando multiplicador de Sazonalidade...")
mask_test = df_recursive['set'] == 'test'

# Natal : Aumento de 10%
natal_mask = mask_test & (df_recursive['data'].dt.day >= 19) & (df_recursive['data'].dt.day <= 24)
df_recursive.loc[natal_mask, 'demanda'] = df_recursive.loc[natal_mask, 'demanda'] * 1.10

# Ano Novo: Aumento de 5%
ano_novo_mask = mask_test & (df_recursive['data'].dt.day >= 29) & (df_recursive['data'].dt.day <= 31)
df_recursive.loc[ano_novo_mask, 'demanda'] = df_recursive.loc[ano_novo_mask, 'demanda'] * 1.05

print("Processo finalizado!")

## 7. Consolidação e Geração da Submissão Final

Nesta etapa conclusiva, isolam-se as previsões geradas para o conjunto de teste. Para garantir a integridade da estrutura exigida pela competição, realiza-se um *merge* entre as previsões calculadas e o arquivo de submissão base (`test.csv`), utilizando as chaves compostas (`data`, `sku`, `cod_filial`) como referência.

Este procedimento assegura que cada `id` original receba sua respectiva demanda prevista, mantendo a ordenação correta. Por fim, o arquivo CSV é exportado contendo apenas as colunas `id` e `demanda`, pronto para ser submetido à plataforma.

In [ ]:
# Filtrar apenas o set de teste já preenchido
df_final = df_recursive[df_recursive['set'] == 'test'].copy()

# Merge com o arquivo de submissão original
submission_base = pd.read_csv('data/test.csv')
submission_base['data'] = pd.to_datetime(submission_base['data'])

# Selecionamos apenas as colunas chave e a demanda prevista
df_final_clean = df_final[['data', 'sku', 'cod_filial', 'demanda']]

# Merge
submission = submission_base.merge(df_final_clean, on=['data', 'sku', 'cod_filial'], how='left')

# Exportação
filename = 'submission.csv'
final_sub = submission[['id', 'demanda']]
final_sub.to_csv(filename, index=False)

print(f"Arquivo '{filename}' gerado com {len(final_sub)} linhas.")
print(final_sub.head())